In [1]:
import pandas as pd
val_df = pd.read_csv(r'C:\Users\prasr\Documents\Northeastern\Sem2\Stanford Question Answering Dataset - PROJECT\Question-Answering-model\data\interim\val_data_with_end_token.csv')
val_df.drop('Unnamed: 0',axis=1,inplace = True)
val_df.head(5)

,id,title,context,question,answer,answer_start,is_impossible,answer_end
0,56ddde6b9a695914005b9628,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France,159,False,165
1,56ddde6b9a695914005b9629,Normans,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,10th and 11th centuries,94,False,117
2,56ddde6b9a695914005b9629,Normans,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,in the 10th and 11th centuries,87,False,117
3,56ddde6b9a695914005b962a,Normans,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,"Denmark, Iceland and Norway",256,False,283
4,56ddde6b9a695914005b962b,Normans,The Normans (Norman: Nourmands; French: Norman...,Who was the Norse leader?,Rollo,308,False,313


In [2]:
val_contexts, val_questions = val_df['context'].tolist(),val_df['question'].tolist()
val_answers = [{'text': ans, 'answer_start':ans_start, 'answer_end':ans_end} 
                 for ans,ans_start,ans_end in 
                 zip(val_df['answer'].tolist(),val_df['answer_start'].tolist(),val_df['answer_end'].tolist())] 
val_answers[0]

{'text': 'France', 'answer_start': 159, 'answer_end': 165}

In [3]:
model_path = r'C:\Users\prasr\Documents\Northeastern\Sem2\Stanford Question Answering Dataset - PROJECT\Question-Answering-model\models'

In [4]:
from transformers import DistilBertTokenizerFast
distil_path = model_path + r'\distilbert_custom'
distil_tokenizer = DistilBertTokenizerFast.from_pretrained(distil_path)

C:\Users\prasr\anaconda3\envs\temp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from transformers import BertTokenizerFast
bert_path = model_path+r'\bert_custom'
bert_tokenizer = BertTokenizerFast.from_pretrained(bert_path)

In [6]:
from torch.utils.data import DataLoader
from transformers import AdamW
import torch
torch.cuda.is_available()

True

In [7]:
from transformers import DistilBertForQuestionAnswering
distil_model = DistilBertForQuestionAnswering.from_pretrained(distil_path)

In [8]:
#there are different heads on top of traditional bert, we are here using QuestionAnswering head
from transformers import BertForQuestionAnswering
bert_model = BertForQuestionAnswering.from_pretrained(bert_path)

In [19]:
device = torch.device('cuda') if torch.cuda.is_available() else torc.device('cpu')
bert_model.to(device)
distil_model.to(device)
# optim = AdamW(model.parameters(),lr=5e-5)

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            

In [20]:
def add_token_positions(encodings,answers,tokenizer):
    start_pos, end_pos = [], []
    for i in range(len(answers)):
        start_pos.append(encodings.char_to_token(i,answers[i]['answer_start']))
        end_pos.append(encodings.char_to_token(i,answers[i]['answer_end']))
        if start_pos[-1] is None:
            start_pos[-1] = tokenizer.model_max_length
        go_back = 1
        while end_pos[-1] is None:
            end_pos[-1] = encodings.char_to_token(i,answers[i]['answer_end']-go_back)
            go_back+=1
        
    encodings.update({
        'start_positions': start_pos,
        'end_positions': end_pos
    })


In [21]:
bert_val_encodings = bert_tokenizer(val_contexts, val_questions, truncation = True, padding = True)
add_token_positions(bert_val_encodings,val_answers,bert_tokenizer)
distil_val_encodings = distil_tokenizer(val_contexts, val_questions, truncation = True, padding = True)
add_token_positions(distil_val_encodings,val_answers,distil_tokenizer)

In [24]:
def predict(tokenizer,model, input_ids, attention_mask):
    model.eval()
    input_ids = torch.tensor(input_ids).reshape(1,-1).to(device)
    attention_mask = torch.tensor(attention_mask).reshape(1,-1).to(device)
#     print(input_ids.shape)
    output = model(input_ids,attention_mask = attention_mask)
    start_scores = output['start_logits']
    end_scores = output['end_logits']
    return start_scores,end_scores
    

In [25]:
# val_contexts,val_questions
counter = 0
for bert_input_ids, bert_attention_mask, distil_input_ids, distil_attention_mask, context, question in zip(bert_val_encodings['input_ids'],bert_val_encodings['attention_mask'], distil_val_encodings['input_ids'],distil_val_encodings['attention_mask'],val_contexts,val_questions):
    bert_start_scores, bert_end_scores = predict(bert_tokenizer,bert_model,bert_input_ids,bert_attention_mask)
    distil_start_scores, distil_end_scores = predict(distil_tokenizer,distil_model,distil_input_ids,distil_attention_mask)
    start_scores, end_scores = (bert_start_scores + distil_start_scores)/2 , (bert_end_scores + distil_end_scores)/2 
    start_idx = torch.argmax(start_scores)
    end_idx = torch.argmax(end_scores)
    tokens = distil_tokenizer.convert_ids_to_tokens(distil_input_ids)
    
#     answer = ' '.join(tokens[start_idx:end_idx+1])
    answer = tokens[start_idx]
    # Select the remaining answer tokens and join them with whitespace.
    for i in range(start_idx + 1, end_idx + 1):
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
#     print(start_idx,end_idx)
    print(f'Context {context}')
    print(f'question {question}')
    print(f'answer {answer}')
    print('-'*30)
    if counter>5:
        break
    counter+=1

Context The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
question In what country is Normandy located?
answer france .
------------------------------
Context The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a

In [26]:
# val_contexts,val_questions
counter = 0
answer_list = []
for bert_input_ids, bert_attention_mask, distil_input_ids, distil_attention_mask, context, question in zip(bert_val_encodings['input_ids'],bert_val_encodings['attention_mask'], distil_val_encodings['input_ids'],distil_val_encodings['attention_mask'],val_contexts,val_questions):
    bert_start_scores, bert_end_scores = predict(bert_tokenizer,bert_model,bert_input_ids,bert_attention_mask)
    distil_start_scores, distil_end_scores = predict(distil_tokenizer,distil_model,distil_input_ids,distil_attention_mask)
    start_scores, end_scores = (bert_start_scores + distil_start_scores)/2 , (bert_end_scores + distil_end_scores)/2 
    start_idx = torch.argmax(start_scores)
    end_idx = torch.argmax(end_scores)
    tokens = distil_tokenizer.convert_ids_to_tokens(distil_input_ids)
    
    answer = tokens[start_idx]
    # Select the remaining answer tokens and join them with whitespace.
    for i in range(start_idx + 1, end_idx + 1):
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
    answer_list.append(answer)

In [27]:
answer_list

['france .',
 '10th and 11th centuries',
 '10th and 11th centuries',
 'denmark , iceland and norway',
 'rollo ,',
 '10th',
 '10th',
 '10th',
 'william the conqueror ,',
 'richard i',
 'catholic orthodoxy',
 'catholic orthodoxy',
 'norseman , viking "',
 'norseman , viking "',
 '9th century )',
 '911',
 'king charles iii',
 'river seine ,',
 'river seine ,',
 "rollo '",
 'unique',
 'north',
 'north',
 'fighting horsemen',
 'seljuk turks .',
 'seljuk turks .',
 'seljuk turks .',
 '1050s .',
 '1050s .',
 '1060s ,',
 '1060s ,',
 'alexius komnenos .',
 'afranji ,',
 'oursel',
 'turkish forces',
 'norman mercenary origin during the period of the comnenian restoration , when byzantine emperors were seeking out western european warriors . the raoulii were descended from an italo - norman',
 'norman mercenary origin during the period of the comnenian restoration , when byzantine emperors were seeking out western european warriors . the raoulii were descended from an italo - norman',
 'norman me

In [28]:
val_df['ensemble_predicted_ans'] = answer_list
val_df.head(5)

,id,title,context,question,answer,answer_start,is_impossible,answer_end,ensemble_predicted_ans
0,56ddde6b9a695914005b9628,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France,159,False,165,france .
1,56ddde6b9a695914005b9629,Normans,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,10th and 11th centuries,94,False,117,10th and 11th centuries
2,56ddde6b9a695914005b9629,Normans,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,in the 10th and 11th centuries,87,False,117,10th and 11th centuries
3,56ddde6b9a695914005b962a,Normans,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,"Denmark, Iceland and Norway",256,False,283,"denmark , iceland and norway"
4,56ddde6b9a695914005b962b,Normans,The Normans (Norman: Nourmands; French: Norman...,Who was the Norse leader?,Rollo,308,False,313,"rollo ,"


In [30]:
ensemble_answers = {}
for index,row in val_df.iterrows():
    ensemble_answers[row['id']] = row['ensemble_predicted_ans']

In [31]:
ensemble_answers

{'56ddde6b9a695914005b9628': 'france .',
 '56ddde6b9a695914005b9629': '10th and 11th centuries',
 '56ddde6b9a695914005b962a': 'denmark , iceland and norway',
 '56ddde6b9a695914005b962b': 'rollo ,',
 '56ddde6b9a695914005b962c': '10th',
 '56dddf4066d3e219004dad5f': 'william the conqueror ,',
 '56dddf4066d3e219004dad60': 'richard i',
 '56dddf4066d3e219004dad61': 'catholic orthodoxy',
 '56dde0379a695914005b9636': 'norseman , viking "',
 '56dde0379a695914005b9637': '9th century )',
 '56dde0ba66d3e219004dad75': '911',
 '56dde0ba66d3e219004dad76': 'king charles iii',
 '56dde0ba66d3e219004dad77': 'river seine ,',
 '56dde1d966d3e219004dad8d': "rollo '",
 '56dde27d9a695914005b9651': 'unique',
 '56dde27d9a695914005b9652': 'north',
 '56dde2fa66d3e219004dad9b': 'fighting horsemen',
 '56de0f6a4396321400ee257f': 'seljuk turks .',
 '56de0ffd4396321400ee258d': '1050s .',
 '56de0ffd4396321400ee258e': '1060s ,',
 '56de0ffd4396321400ee258f': 'alexius komnenos .',
 '56de10b44396321400ee2593': 'afranji ,',


In [32]:
import json
output_file = r'C:\Users\prasr\Documents\Northeastern\Sem2\Stanford Question Answering Dataset - PROJECT\Question-Answering-model\data\processed\ensemble_results.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(ensemble_answers, f, ensure_ascii=False, indent=4)